In [ ]:
import numpy as np
import tensorflow as tf
import gpflow as gpf
import matplotlib.pyplot as plt
from core.objectives import sample_GP_prior_utilities

In [ ]:
seed = 0
rng = np.random.default_rng(seed)
tf.random.set_seed(seed)
kernel = gpf.kernels.SquaredExponential(lengthscales=[0.1, 0.1])
lowers = [0., 0.]
uppers = [1., 1.]
num_agents = 2
num_actions = 8

In [ ]:
u = sample_GP_prior_utilities(num_agents,
                              kernel,
                              lowers,
                              uppers,
                              num_points=100,
                              rng=rng)

In [ ]:
from core.utils import combinations
from pathlib import Path

def plot_utilities_2d(u,
                      xlims,
                      ylims,
                      num_actions,
                      title="",
                      cmap="Spectral",
                      save=False,
                      save_dir="",
                      filename="",
                      show_plot=True):
    xmin, xmax = xlims
    ymin, ymax = ylims
    xdomain = np.linspace(xmin, xmax, num_actions)[:, None]
    ydomain = np.linspace(ymin, ymax, num_actions)[:, None]
    combs = combinations(xdomain, ydomain)
    xlabel = 'Agent 1 actions'
    ylabel = 'Agent 2 actions'

    u1_vals = u[0](combs)
    u1_reshaped = np.transpose(
        np.reshape(u1_vals, [num_actions, num_actions]))
    u2_vals = u[1](combs)
    u2_reshaped = np.transpose(
        np.reshape(u2_vals, [num_actions, num_actions]))

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title, size=20)
    fig.set_size_inches(8, 4)
    fig.set_dpi(200)

    im1 = ax1.imshow(u1_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax1.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax1.plot(*mean_stationary, marker='*', markersize=20, color='white')
    ax1.set_title("Agent 1 utility", size=16)
    ax1.set_xlabel(xlabel, size=12)
    ax1.set_ylabel(ylabel, size=12)
    fig.colorbar(im1, ax=ax1)

    im2 = ax2.imshow(u2_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax2.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax2.plot(*f_std_argmin, marker='*', markersize=20, color='white')
    ax2.set_title("Agent 2 utility", size=16)
    ax2.set_xlabel(xlabel, size=12)
    ax2.set_ylabel(ylabel, size=12)
    fig.colorbar(im2, ax=ax2)
    fig.tight_layout()

    if save:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(save_dir + filename, bbox_inches='tight')

    if show_plot:
        plt.show()

In [ ]:
plot_utilities_2d(u,
                  (lowers[0], uppers[0]),
                  (lowers[1], uppers[1]),
                  num_actions)

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
xdomain = np.linspace(xmin, xmax, num_actions)[:, None]
ydomain = np.linspace(ymin, ymax, num_actions)[:, None]
combs = combinations(xdomain, ydomain)
actions = np.squeeze(xdomain)

In [ ]:
def join_action(s,
                i,
                actions):
    """
    Given a strategy profile s, fixes the actions of all other agents, and replaces the action of the i-th agent
    with all possible actions.
    :param s: Array of shape (N, ). Strategy profile.
    :param i: int. Agent.
    :param actions: Array of shape (M, ). All possible actions.
    :return: Array of shape (M, N).
    """
    M = len(actions)
    joined = np.tile(s[None, :], (M, 1))
    joined[:, i] = actions
    return joined

In [ ]:
def best_response_payoff_pure(u,
                              S,
                              actions):
    """
    Calculates the best response payoff for each pure strategy profile in S, for each agent.
    :param u: List of utility functions.
    :param S: array of shape (M ** N, N). All possible pure strategy profiles of the N agents.
    :param actions: array of shape (M, ). All possible M actions.
    :return: array of shape (M ** N, N).
    """
    M = len(actions)
    _, N = S.shape
    brp = np.zeros((M ** N, N))

    for i in range(N):
        for j in range(len(S)):
            s = S[j]
            joined = join_action(s, i, actions)
            best_util = np.max(u[i](joined))
            current_util = u[i](s[None, :])
            brp[j, i] = best_util - current_util
    
    return brp

In [ ]:
brp = best_response_payoff_pure(u, combs, actions)

In [ ]:
def plot_brp_2d(brp,
                xlims,
                ylims,
                actions,
                title="",
                cmap="Spectral",
                save=False,
                save_dir="",
                filename="",
                show_plot=True):
    xmin, xmax = xlims
    ymin, ymax = ylims
    num_actions = len(actions)
    xlabel = 'Agent 1 actions'
    ylabel = 'Agent 2 actions'
    
    brp_max = np.max(brp, axis=1)
    brp_max_reshaped = np.transpose(
        np.reshape(brp_max, [num_actions, num_actions]))

    fig, (ax1) = plt.subplots(1, 1)
    fig.suptitle(title, size=12)
    fig.set_size_inches(6, 3)
    fig.set_dpi(200)

    im1 = ax1.imshow(brp_max_reshaped,
                     interpolation='nearest',
                     extent=(xmin, xmax, ymin, ymax),
                     origin='lower',
                     cmap=cmap,
                     aspect=(xmax - xmin) / (ymax - ymin))
    # ax1.plot(inputs[:, 0], inputs[:, 1], 'ko', mew=2)
    # ax1.plot(*mean_stationary, marker='*', markersize=20, color='white')
    ax1.set_title("Max best response payoff", size=12)
    ax1.set_xlabel(xlabel, size=8)
    ax1.set_ylabel(ylabel, size=8)
    fig.colorbar(im1, ax=ax1)
    
    fig.tight_layout()
    if save:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(save_dir + filename, bbox_inches='tight')
    if show_plot:
        plt.show()

In [ ]:
np.min(np.max(brp, axis=1))

In [ ]:
plot_brp_2d(brp, 
            (lowers[0], uppers[0]),   
            (lowers[1], uppers[1]),
            actions)

In [ ]:
import numpy as np
import tensorflow as tf
import gpflow as gpf
from core.objectives import sample_GP_prior_utilities, noisy_observer
from core.models import create_models
from core.utils import cross_product


seed = 0
rng = np.random.default_rng(seed)
tf.random.set_seed(seed)
num_agents = 2
num_actions = 8
ls = np.array([0.1] * num_agents)
lowers = [0.] * num_agents
uppers = [1.] * num_agents
noise_variance = 0.1
num_init_points = 30

actions = np.linspace(lowers[0], uppers[0], num_actions)
domain = actions[:, None]
for i in range(1, num_agents):
    next_actions = np.linspace(lowers[i], uppers[i], num_actions)
    domain = cross_product(domain, next_actions[:, None])

kernel = gpf.kernels.SquaredExponential(lengthscales=ls)
u = sample_GP_prior_utilities(num_agents,
                              kernel,
                              lowers,
                              uppers,
                              num_points=100,
                              rng=rng)
observer = noisy_observer(u=u,
                          noise_variance=noise_variance,
                          rng=rng)
init_idxs = rng.integers(low=0, high=num_actions ** num_agents, size=num_init_points)
init_X = domain[init_idxs]
init_data = (init_X, observer(init_X))

models = create_models(data=init_data,
                       kernel=kernel,
                       noise_variance=noise_variance)


In [ ]:
import matplotlib.pyplot as plt

def plot_models_2d(models,
                   xlims,
                   ylims,
                   actions,
                   domain,
                   X=None,
                   title="",
                   cmap="Spectral",
                   save=False,
                   save_dir="",
                   filename="",
                   show_plot=True):
    xmin, xmax = xlims
    ymin, ymax = ylims
    num_actions = len(actions)
    xlabel = 'Agent 1 actions'
    ylabel = 'Agent 2 actions'

    g1_mean, g1_var = models[0].predict_f(domain)
    g1_mean_reshaped = np.transpose(
        np.reshape(g1_mean, [num_actions, num_actions]))
    g1_var_reshaped = np.transpose(
        np.reshape(g1_var, [num_actions, num_actions]))
    g2_mean, g2_var = models[1].predict_f(domain)
    g2_mean_reshaped = np.transpose(
        np.reshape(g2_mean, [num_actions, num_actions]))
    g2_var_reshaped = np.transpose(
        np.reshape(g2_var, [num_actions, num_actions]))

    fig, axs = plt.subplots(2, 2)
    fig.suptitle(title, size=10)
    fig.set_size_inches(5, 4)
    fig.set_dpi(200)
    
    offset = (1/num_actions) / 2
    for i in range(2):
        for j in range(2):
            if i == 0:
                if j == 0:
                    f = g1_mean_reshaped
                    title = "Agent 1 GP mean"
                elif j == 1:
                    f = g2_mean_reshaped
                    title = "Agent 2 GP mean"
            elif i == 1:
                if j == 0:
                    f = g1_var_reshaped
                    title = "Agent 1 GP variance"
                elif j == 1:
                    f = g2_var_reshaped
                    title = "Agent 2 GP variance"
            ax = axs[i, j]
            im = ax.imshow(f,
                           interpolation='None',
                           extent=(xmin-offset, xmax+offset, ymin-offset, ymax+offset),
                           origin='lower',
                           cmap=cmap,
                           aspect=(xmax - xmin) / (ymax - ymin))
            if X is not None:
                ax.plot(X[:, 0], X[:, 1], 'x', mew=1, c='black')
            ax.set_title(title, size=8)
            ax.set_xlabel(xlabel, size=8)
            ax.set_ylabel(ylabel, size=8)
            fig.colorbar(im, ax=ax)

    fig.tight_layout()
    if save:
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        plt.savefig(save_dir + filename, bbox_inches='tight')
    if show_plot:
        plt.show()


In [ ]:
plot_models_2d(models, 
            (lowers[0], uppers[0]),   
            (lowers[1], uppers[1]),
            actions,
               domain,
              domain[:8])

In [ ]:
from metrics.plotting import plot_utilities_2d

In [ ]:
plot_utilities_2d(u,
                  (lowers[0], uppers[0]),
                  (lowers[1], uppers[1]),
                  actions)

In [ ]:
models[0].predict_f(combs)

In [ ]:
xdomain = actions[:, None]
ydomain = actions[:, None]
num_actions = len(actions)
combs = combinations(xdomain, ydomain)

In [ ]:
combs

In [ ]:
def cross_product(x, y):
    """

    :param x: array of shape (m, d_x)
    :param y: array of shape (n, d_y)
    :return:  array of shape (m * n, d_x + d_y)
    """
    m, d_x = x.shape
    n, d_y = y.shape
    x_temp = np.tile(x[:, :, None], (1, n, 1))
    x_temp = np.reshape(x_temp, [m * n, d_x])
    y_temp = np.tile(y, (m, 1))
    return np.concatenate([x_temp, y_temp], axis=-1)

In [ ]:
np.allclose(cross_product(xdomain, ydomain), combs)

In [42]:
actions = np.linspace(lowers[0], uppers[0], num_actions)
domain = actions[:, None]
for i in range(1, num_agents):
    next_actions = np.linspace(lowers[i], uppers[i], num_actions)
    domain = cross_product(domain, next_actions[:, None])

In [43]:
domain

array([[0.        , 0.        ],
       [0.        , 0.14285714],
       [0.        , 0.28571429],
       [0.        , 0.42857143],
       [0.        , 0.57142857],
       [0.        , 0.71428571],
       [0.        , 0.85714286],
       [0.        , 1.        ],
       [0.14285714, 0.        ],
       [0.14285714, 0.14285714],
       [0.14285714, 0.28571429],
       [0.14285714, 0.42857143],
       [0.14285714, 0.57142857],
       [0.14285714, 0.71428571],
       [0.14285714, 0.85714286],
       [0.14285714, 1.        ],
       [0.28571429, 0.        ],
       [0.28571429, 0.14285714],
       [0.28571429, 0.28571429],
       [0.28571429, 0.42857143],
       [0.28571429, 0.57142857],
       [0.28571429, 0.71428571],
       [0.28571429, 0.85714286],
       [0.28571429, 1.        ],
       [0.42857143, 0.        ],
       [0.42857143, 0.14285714],
       [0.42857143, 0.28571429],
       [0.42857143, 0.42857143],
       [0.42857143, 0.57142857],
       [0.42857143, 0.71428571],
       [0.

In [46]:
np.where(domain == domain[1:2])

(array([ 0,  1,  1,  2,  3,  4,  5,  6,  7,  9, 17, 25, 33, 41, 49, 57],
       dtype=int64),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int64))

In [47]:
domain[1:2]

array([[0.        , 0.14285714]])

In [58]:
def index(array, item):
    for idx, val in enumerate(array):
        if np.allclose(val, item):
            return idx

In [60]:
index(domain, domain[60])

60

In [54]:
domain[1:2]

array([[0.        , 0.14285714]])